# Data cleaning
We want the class with labels as numerical value and the body with clean text.

This will remove:
* duplicates
* NaN entires
* non english
* url, html

* make it lowercase
* combine title and body

In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append("../../scripts_shared/")
from preprocess_text import preprocess_text


In [2]:
# Read CSV into a dataframe
#high_priority = pd.read_csv("csv/high_priority_no_td.csv", index_col=0)
medium_priority = pd.read_csv("csv/medium_priority_no_td.csv", index_col=0)
#low_priority = pd.read_csv("csv/low_priority_no_td.csv", index_col=0)
not_priority = pd.read_csv("csv/not_priority.csv", index_col=0)

/tmp/ipykernel_163488/4088307971.py:5: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  not_priority = pd.read_csv("csv/not_priority.csv", index_col=0)


In [3]:
# Number or different labels
medium_priority.labels.value_counts().to_frame()[:50]

,count
labels,
auto-migrated Priority-Medium Type-Defect,21756
Medium Priority,3239
Priority: Medium,2875
bug imported Priority-Medium,2712
auto-migrated Priority-Medium Type-Enhancement,2069
medium priority,1938
bug priority: medium,1936
enhancement medium priority,1814
migrated Priority: Medium Type: Bug,1588


In [4]:
#Give each priority a label by number.
# 'Label encoding'. Makes is easier for machine learning models to work with categorical data.
medium_priority["labels"] = 0
medium_priority["class"] = "medium_priority"
not_priority["labels"] = 1
not_priority["class"] = "not_priority"

In [5]:
priority = pd.concat([medium_priority, not_priority] , ignore_index = True)

In [6]:
priority[priority["repo"] == "python/mypy"]

,id,type,created_at,repo,repo_url,action,title,labels,body,class
257345,1.142706e+10,IssuesEvent,2020-02-03 23:33:27,python/mypy,https://api.github.com/repos/python/mypy,closed,Type of conditional expression is object,1,The type of the following conditional expressi...,not_priority
257374,1.142707e+10,IssuesEvent,2020-02-03 23:35:24,python/mypy,https://api.github.com/repos/python/mypy,closed,Type inference of Tuples returns object instead,1,While interacting with `zip` I encountered a v...,not_priority
272842,1.362788e+10,IssuesEvent,2020-09-24 13:13:03,python/mypy,https://api.github.com/repos/python/mypy,opened,Daemon support for --follow-imports=silent,1,"After #5870 is done, it would be nice to also ...",not_priority
281360,1.363241e+10,IssuesEvent,2020-09-24 19:36:27,python/mypy,https://api.github.com/repos/python/mypy,opened,regression: assignment of 'builtins.type' now ...,1,\r\n**Bug Report**\r\n\r\n#7963 causes a serio...,not_priority
298892,2.407484e+10,IssuesEvent,2022-09-18 17:03:35,python/mypy,https://api.github.com/repos/python/mypy,opened,stubtest: more concise error for forgotten arg...,1,**Feature**\r\n\r\nWhen the implementation add...,not_priority
...,...,...,...,...,...,...,...,...,...,...
16630437,7.349652e+09,IssuesEvent,2018-03-08 11:27:27,python/mypy,https://api.github.com/repos/python/mypy,closed,Legal Python circular imports result in loss o...,1,Example:\r\n\r\n`test/__init__.py`:\r\n```pyth...,not_priority
16630438,7.349652e+09,IssuesEvent,2018-03-08 11:27:27,python/mypy,https://api.github.com/repos/python/mypy,closed,Crash when serializing property with forward r...,1,This serialization test case fails with a cras...,not_priority
16631213,7.349972e+09,IssuesEvent,2018-03-08 12:44:13,python/mypy,https://api.github.com/repos/python/mypy,opened,Suggest type annotations for partial types spa...,1,We don't support inferring partial types from ...,not_priority
16631226,7.349979e+09,IssuesEvent,2018-03-08 12:45:41,python/mypy,https://api.github.com/repos/python/mypy,closed,What to do about inferring types from multiple...,1,Typically mypy infers the type of the variable...,not_priority


In [7]:
# Remove mypy from the dataset
priority = priority[priority["repo"] != "python/mypy"]
priority

,id,type,created_at,repo,repo_url,action,title,labels,body,class
0,1.141757e+10,IssuesEvent,2020-02-03 00:11:29,poissonconsulting/nlist,https://api.github.com/repos/poissonconsulting...,closed,"tidy.nlists should return table with term, est...",0,NaN,medium_priority
1,1.141781e+10,IssuesEvent,2020-02-03 01:38:13,ericauv/ericauv-portfolio,https://api.github.com/repos/ericauv/ericauv-p...,closed,Increase Indent of ListItem,0,ListItems should have greater indent to differ...,medium_priority
2,1.141805e+10,IssuesEvent,2020-02-03 02:48:17,poissonconsulting/nlist,https://api.github.com/repos/poissonconsulting...,closed,deprecate as.nlist and as.nlists for as_nlist ...,0,NaN,medium_priority
3,1.141809e+10,IssuesEvent,2020-02-03 03:00:58,poissonconsulting/nlist,https://api.github.com/repos/poissonconsulting...,opened,Add as_mcmc and as_mcmc_list wrappers on as.mc...,0,"but\r\n```\r\n> as_mcmc(nlist(x = matrix(1:6, ...",medium_priority
4,1.141821e+10,IssuesEvent,2020-02-03 03:33:19,poissonconsulting/nlist,https://api.github.com/repos/poissonconsulting...,opened,Add as.nlist.mcmc and as.nlists.mcmc,0,NaN,medium_priority
...,...,...,...,...,...,...,...,...,...,...
16649544,8.109961e+09,IssuesEvent,2018-08-14 09:21:15,urbit/arvo,https://api.github.com/repos/urbit/arvo,opened,%bad-text trips up hall JSON conversion,1,"Haven't tested this in detail yet, but pretty ...",not_priority
16649545,8.109961e+09,IssuesEvent,2018-08-14 09:21:16,highcharts/highcharts-react,https://api.github.com/repos/highcharts/highch...,closed,HighMaps mapBubble type,1,"Hello,\r\nIt is possible to create also HighMa...",not_priority
16649546,8.109982e+09,IssuesEvent,2018-08-14 09:25:07,Loriowar/comindivion,https://api.github.com/repos/Loriowar/comindivion,opened,Add a validation on a belonging of a predicate...,1,Now user can change html content of the intera...,not_priority
16649547,8.109983e+09,IssuesEvent,2018-08-14 09:25:24,TiiQu-Network/TQ-test-page,https://api.github.com/repos/TiiQu-Network/TQ-...,closed,Error on submit with no values,1,TypeError: Too few arguments in function sum (...,not_priority


In [8]:
# Drop duplicates by the content of the body
priority = priority.drop_duplicates(subset=['title'], keep='last')
priority.dropna(inplace=True)
priority.reset_index(inplace=True)
priority.drop(columns=["index"] , inplace= True)
priority["class"].value_counts()

/tmp/ipykernel_163488/2859834588.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority.dropna(inplace=True)
/tmp/ipykernel_163488/2859834588.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority.drop(columns=["index"] , inplace= True)


class
not_priority       1356577
medium_priority     154206
Name: count, dtype: int64

In [9]:
# Split the dataframe 
medium_priority = priority.loc[priority['class'] == 'medium_priority'].copy()
not_priority = priority.loc[priority['class'] == 'not_priority'].copy()

In [10]:
medium_label_counts = medium_priority["class"].value_counts()
not_label_counts = not_priority["class"].value_counts()
mp_count = medium_label_counts["medium_priority"]
print(f"med count: {mp_count}")
not_count = not_label_counts["not_priority"]
print(f"not count: {not_count}")

med count: 154206
not count: 1356577


In [11]:
not_priority = not_priority.sample(frac=mp_count/not_count, random_state=42)
med_vs_not = pd.concat([medium_priority, not_priority] , ignore_index = True)
med_vs_not["class"].value_counts()

class
medium_priority    154206
not_priority       154206
Name: count, dtype: int64

In [12]:
print(med_vs_not["title"][0])
print(med_vs_not["body"][0])

Increase Indent of ListItem
ListItems should have greater indent to differentiate them from ListTitle


In [13]:
# Copy content of body to a new col named text
med_vs_not["text"] = med_vs_not["title"] + med_vs_not["body"]
med_vs_not.tail()

,id,type,created_at,repo,repo_url,action,title,labels,body,class,text
308407,1.051316e+10,IssuesEvent,2019-09-27 19:49:05,olifolkerd/tabulator,https://api.github.com/repos/olifolkerd/tabulator,closed,No Scrollbar on the Table when virtualDom is s...,1,**Describe the bug**\r\n\r\n when setting `vir...,not_priority,No Scrollbar on the Table when virtualDom is s...
308408,9.307850e+09,IssuesEvent,2019-03-25 13:20:02,Microsoft/vscode,https://api.github.com/repos/Microsoft/vscode,closed,New explorer: clicking on a file first shows b...,1,I feel like this is different behavior from be...,not_priority,New explorer: clicking on a file first shows b...
308409,1.468090e+10,IssuesEvent,2020-12-31 11:32:42,MichaelHegner/p2p-statistic,https://api.github.com/repos/MichaelHegner/p2p...,closed,Separate Earning and Costs,1,Currently Costs and Earnings are aggregated. T...,not_priority,Separate Earning and CostsCurrently Costs and ...
308410,1.362260e+10,IssuesEvent,2020-09-24 04:07:14,dlab-berkeley/BashGit,https://api.github.com/repos/dlab-berkeley/Bas...,opened,Provide a real world example,1,Something like a shell script that creates a d...,not_priority,Provide a real world example Something like a ...
308411,1.363214e+10,IssuesEvent,2020-09-24 19:09:17,test-organization-kkjeer/bot-validation,https://api.github.com/repos/test-organization...,closed,Auto monkey issue a1tfh8,1,Issue: switch label upon author reply\nvia `is...,not_priority,Auto monkey issue a1tfh8Issue: switch label up...


In [14]:
med_vs_not["text"][0]

'Increase Indent of ListItemListItems should have greater indent to differentiate them from ListTitle'

In [15]:
# Make a new dataframe with only text, label and class cols.
all_priority_subset = med_vs_not[["text" , "labels" , "class"]]
all_priority_subset

,text,labels,class
0,Increase Indent of ListItemListItems should ha...,0,medium_priority
1,Add as_mcmc and as_mcmc_list wrappers on as.mc...,0,medium_priority
2,Create New Option For Dropdown Scriptable Obje...,0,medium_priority
3,Header takes too much vertical space on wide s...,0,medium_priority
4,PageBody extends past html containerWhen navig...,0,medium_priority
...,...,...,...
308407,No Scrollbar on the Table when virtualDom is s...,1,not_priority
308408,New explorer: clicking on a file first shows b...,1,not_priority
308409,Separate Earning and CostsCurrently Costs and ...,1,not_priority
308410,Provide a real world example Something like a ...,1,not_priority


In [16]:
# Convert to string
all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)

/tmp/ipykernel_163488/3106036705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)


In [17]:
all_priority_subset

,text,labels,class,text_str
0,Increase Indent of ListItemListItems should ha...,0,medium_priority,Increase Indent of ListItemListItems should ha...
1,Add as_mcmc and as_mcmc_list wrappers on as.mc...,0,medium_priority,Add as_mcmc and as_mcmc_list wrappers on as.mc...
2,Create New Option For Dropdown Scriptable Obje...,0,medium_priority,Create New Option For Dropdown Scriptable Obje...
3,Header takes too much vertical space on wide s...,0,medium_priority,Header takes too much vertical space on wide s...
4,PageBody extends past html containerWhen navig...,0,medium_priority,PageBody extends past html containerWhen navig...
...,...,...,...,...
308407,No Scrollbar on the Table when virtualDom is s...,1,not_priority,No Scrollbar on the Table when virtualDom is s...
308408,New explorer: clicking on a file first shows b...,1,not_priority,New explorer: clicking on a file first shows b...
308409,Separate Earning and CostsCurrently Costs and ...,1,not_priority,Separate Earning and CostsCurrently Costs and ...
308410,Provide a real world example Something like a ...,1,not_priority,Provide a real world example Something like a ...


In [18]:
# Clean the data.
all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)

/tmp/ipykernel_163488/63294665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)


In [19]:
# Make a subset with text_clean and label
priority_label_text = all_priority_subset[["text_clean" , "labels", "class"]]
priority_label_text

,text_clean,labels,class
0,increase indent of listitemlistitems should ha...,0,medium_priority
1,add asmcmc and asmcmclist wrappers on asmcmc a...,0,medium_priority
2,create new option for dropdown scriptable obje...,0,medium_priority
3,header takes too much vertical space on wide s...,0,medium_priority
4,pagebody extends past html containerwhen navig...,0,medium_priority
...,...,...,...
308407,no scrollbar on the table when virtualdom is s...,1,not_priority
308408,new explorer clicking on a file first shows bl...,1,not_priority
308409,separate earning and costscurrently costs and ...,1,not_priority
308410,provide a real world example something like a ...,1,not_priority


In [20]:
# Need to dropna here since cleaning function returns NaN for not english text.
priority_label_text.dropna(inplace=True)
priority_label_text.reset_index(inplace=True)
priority_label_text.drop(columns=["index"] , inplace= True)

priority_label_text

/tmp/ipykernel_163488/239147930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.dropna(inplace=True)
/tmp/ipykernel_163488/239147930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.drop(columns=["index"] , inplace= True)


,text_clean,labels,class
0,increase indent of listitemlistitems should ha...,0,medium_priority
1,add asmcmc and asmcmclist wrappers on asmcmc a...,0,medium_priority
2,create new option for dropdown scriptable obje...,0,medium_priority
3,header takes too much vertical space on wide s...,0,medium_priority
4,pagebody extends past html containerwhen navig...,0,medium_priority
...,...,...,...
279284,no scrollbar on the table when virtualdom is s...,1,not_priority
279285,new explorer clicking on a file first shows bl...,1,not_priority
279286,separate earning and costscurrently costs and ...,1,not_priority
279287,provide a real world example something like a ...,1,not_priority


In [21]:
# Clean dataset with clean text and labels.
# 0 = high priority, 1 = not high priority
file_name = f"csv/clean_med_vs_not_priority.csv"
priority_label_text.to_csv(file_name, index=False)

In [22]:
pri = pd.read_csv(file_name)
pri

,text_clean,labels,class
0,increase indent of listitemlistitems should ha...,0,medium_priority
1,add asmcmc and asmcmclist wrappers on asmcmc a...,0,medium_priority
2,create new option for dropdown scriptable obje...,0,medium_priority
3,header takes too much vertical space on wide s...,0,medium_priority
4,pagebody extends past html containerwhen navig...,0,medium_priority
...,...,...,...
279284,no scrollbar on the table when virtualdom is s...,1,not_priority
279285,new explorer clicking on a file first shows bl...,1,not_priority
279286,separate earning and costscurrently costs and ...,1,not_priority
279287,provide a real world example something like a ...,1,not_priority
